# We will be using Tesla stock data and will be checking how effective are SMA and RSI for buying opportunities.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv("../input/tesla-stock-price/Tesla.csv - Tesla.csv.csv")
data

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
data["Date"] = pd.to_datetime(data['Date'])

In [ ]:
plt.figure(figsize = (15,7))
x = data["Date"]
y = data["Close"]
plt.plot(x,y,color = "purple")
plt.grid()

# Simple Moving-Average 

The number of prices within a time period is divided by the number of total periods.
Using SMA indicator is one of the safest stragies for long-term trading but it doesn't work for intraday trading.

In [ ]:
ma_20 = data.rolling(window=20).mean()
ma_100 = data.rolling(window = 100).mean()

In [ ]:
ma_20 = ma_20["Close"].fillna(0)
# for dealing with the values ahead
ma_100 = ma_100["Close"].fillna(0)

In [ ]:
plt.figure(figsize = (15,7))
x = data["Date"]
y = data["Close"]
plt.plot(x,y,color = "black",label = "Price")
plt.plot(x,ma_20,color = "red",label = "MA 20")
plt.plot(x,ma_100,color = "green",label = "MA 100")
plt.grid()
plt.legend()


In [ ]:
data["MA100"] = ma_100

In [ ]:
data["MA20"] = ma_20

Using plotly for better observation of the chart (golden cross).

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot as plt

In [ ]:
px.line(data ,x = "Date",y = ["Close","MA20","MA100"],width = 1200,height = 600)


In [ ]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

fig.show()

 Buying shares whenever a golden cross occurs.

In [ ]:

sum_profit = 0
for i in range(1,len(ma_100)):
    if ma_20[i-1] <= ma_100[i-1] and ma_20[i]>ma_100[i]:
        bPrice = data["Close"][i-1]
        sPrice = data["Close"][i+29]
        pt = 50*(sPrice - bPrice)
#         buying 50 shares and holding it for 30 days
        sum_profit += pt
print(f"Total Profit : ${round(sum_profit)}")
        

In [ ]:
px.line(data ,x = "Date",y = "Volume",width = 1200,height = 600)

# RSI

Formula for RSI = 100 - (100/(1 + Average Gain/Average Loss))

I tried to use diff() but it gave some errors, so I had to take the long way.

In [ ]:
closeprice = data["Close"]
upClose = []
downClose = []
i = 0
while i < len(data["Close"]):
    if i == 0:
        upClose.append(0)
        downClose.append(0)
    else:
        if (closeprice[i] - closeprice[i-1]) > 0:
            upClose.append(closeprice[i] - closeprice[i-1])
            downClose.append(0)
        else:
            downClose.append(closeprice[i] - closeprice[i-1])
            upClose.append(0)
            
    i = i+1

a = 0 
avgGain = []
avgLoss = []
while a < len(upClose):
    if a < 15:
        avgGain.append(0)
        avgLoss.append(0)
    else:
        sumGain = 0
        sumLoss = 0
        b = a - 14
        while b<= a:
            sumGain += upClose[b]
            sumLoss += downClose[b]
            b += 1
        avgGain.append(sumGain/14)
        avgLoss.append(abs(sumLoss/14))
        
    a +=1
    
c = 0 
RS = []
RSI = []
while c < len(data["Close"]):
    if c < 15:
        RS.append(0)
        RSI.append(0)
    else:
        RSvalue = (avgGain[c]/avgLoss[c])
        RS.append(RSvalue)
        RSI.append(100-100/(1+RSvalue))
    c = c+1

 

In [ ]:
data["RSI"] = RSI

In [ ]:
import seaborn as sns
plt.figure(figsize = (25,7))
sns.set_theme(style = "darkgrid")
sns.lineplot(y = "RSI",x ="Date",data = data,color = "red")
sns.lineplot(y = 75,x = "Date",data = data,color = "black")
sns.lineplot(y = 25,x = "Date",data = data,color = "black")

Checking the number of buying opportunities using RSI indicator.
Whenever stock's RSI goes below 25, hence it becomes oversold and gives some buying opportunities.
Therefore buying 50 shares of Tesla and holding it for 10 days.

In [ ]:
b_count = 0
for i in data["RSI"]:
    if i<= 25 and i == 0  :
        b_count += 1
b_count
 

In [ ]:
count = len(data["Close"])
total_profit = 0

for i in range(count):
    if RSI[i]<=25 and RSI[i] == 0 :
        buy_price = data["Close"][i]
        sell_price= data["Close"][i+10]
        profit = 50*(sell_price - buy_price) 
#         buying 50 shares
        total_profit += profit
    
    
print(f"Total Profit : ${round(total_profit)}")
        
